## 二分探索
O(log n)

最小値の最大化、最大値の最小化などの場合にうまく使えることがある   
また、全組み合わせを試すことができない場合（組み合わせ多すぎて間に合わない、そもそも組み合わせに落とし込めない）に、条件を満たすかどうか、適当な値ずつ範囲を狭めて答えを絞っていく方法が考えられるが、その時に使える。  
一度のループで範囲が半分ずつになっていく。

In [ ]:
"""
POJ 1064 Cable master


同じ長さの紐をK本切り出す
・最長でもmax(L)
・長さaを作ると決めたとき、Liから切り出せるのは Li//a 本

aをどの値にすればいいか求める
案１：１～max(L)まで１ずつ増やして確かめる：max(L)*Nより間に合わない
案２：二分探索で決める
"""

N = int(input())
K = int(input())
L = list(map(float, input().split()))

ML= max(L)
lo = 0
hi = ML
# 配列の位置の絞り込みとかなら、while hi - lo > 1　みたいな条件にするが、
# 今回は mid がいい感じの値に（小数第2位まで出力）落ち着けばいいのでそこそこの回数繰り返せばいい
# 一回ごとに範囲が半分になるので、30回もやれば2**30分の1になる
for _ in range(300):
    mid = (lo+hi)/2
    # mid分切り出したときに何本作れるか
    cnt = 0
    for l in L:
        cnt += l // mid
    if cnt >= K:
        lo = mid
    else:
        hi = mid

print(mid)


"""
4
11
8.02 7.43 4.57 5.39
2.005
"""

In [ ]:
"""
POJ 2456 Agressive cows

試行を繰り返して求めるのは難しい（というか無理？）
愚直な方法として
案１：あらゆる牛の配置法を全て試して、それぞれの最短距離の中から最大値を選ぶ：間に合うわけがない
案２：距離１ずつ離して配置できるか試す→２ずつ離して置けるか試す→...　を繰り返して、置けなくなった時の手前の回の距離が最大の最小距離
　　→これも間に合わない（牛の数やら小屋の配置に比例するので）
案２の方法は、１ずつ範囲が絞り込まれてるのが無駄というかボトルネック。一回の施行でもっと答えを絞っていきたい
　　→　二分探索
"""


N = int(input())
M = int(input())
X = list(map(int, input().split()))

X.sort()
lo = 1
hi = X[-1]
mid = (lo+hi)//2
while hi-lo > 1:
    #print(lo, hi, mid)
    cnt = 1
    curr = X[0]
    idx_next = 1
    while idx_next <= len(X)-1:
        if mid <= X[idx_next] - curr:
            cnt += 1
            curr = X[idx_next]
        idx_next += 1
        if cnt == M:
            lo = mid
            break
        elif cnt < M and idx_next >= len(X)-1:
            hi = mid
            break
    mid = (lo+hi)//2
print(mid)


"""
5
3
1 2 8 4 9

"""

In [ ]:
"""
平均最大化

一見 v/w でソートして大きいものから選べばいいようにも思えるが無理。
案１：ｋ個選ぶ方法をすべてためして重さ当たりの価値が最大の物を選ぶ：間に合わない
案２：単位当たり重さが0以上→0.1以上→ ... と試していき、重さ当たり価値の最大を選んでいく：無理（刻みが分からない、時間がかかる）

単位当たり重さ：Σ(v) / Σ(w) >= x 
なるｘについてｘが最大のものを見つけたい
→ Σ(v) - x * Σ(w) >= 0
→ Σ(v - w*x) >= 0
これがある品物の平均に対する寄与度みたいなもので、これが成立する、値の大きいものからｋ個選んだ時にそれらの和が0以上なら或る重さ当たり価値xは作ることができる

"""

N = int(input())
K = int(input())
wv = []
for i in range(N):
    wv.append(tuple(map(int, input().split())))



lo = 0
hi = 10**6
mid = (lo+hi)/2.0
# 今回もインデックスを求めるわけではないので、hi-lo>1 でそれっぽい値に落ち着かない
# 適当な回数繰り返すか、変化量が一定以下になったら止めるとかでいいと思う
for _ in range(30):
    y = [0] * N
    for i in range(N):
        y[i] = wv[i][1] - mid*wv[i][0]

    y.sort(reverse=True)
    # 大きいほうからk個取得
    total = sum(y[:K])
    if total > 0:
        lo = mid
    else:
        hi = mid
    mid = (hi+lo)/2.0

print(mid)


"""
3
2
2 2
5 3
2 1

"""

In [ ]:
"""
AtCoder Regular Contest 037 C



小さいほうからK番目の値がX：X-1以下の値がK個未満しかない＆X以下の値がK個以上
→ X以下の値がK個以上ある最小のXを求める
ここでXはAとBの要素の積からできる。

Xをなんか決めてX以下の値がK個以上ある　のXについて最小を求める
AとBをソートしておく
普通にAxBをすべてやり、それがX以下か確かめると間に合わないので、
Ai*Bj <= X -> Bj <= X // Ai
から、各行について上の式を満たすjの個数（＝その行のAxBにおいてX以下となる個数）を二分探索で求め、全行の個数の合計がK以上となる最小のXを探す

pypyじゃないと間に合わない

"""

from bisect import bisect

N,K = map(int, input().split())

A = list(map(int, input().split()))
B = list(map(int, input().split()))

A.sort()
B.sort()


lo = 0
hi = (10**9)**2

while hi - lo > 1:
    mid = (hi+lo)//2
    # 各行にあるX（変数としてはmid）以下の個数を求める
    cnt = 0
    for i in range(N):
        cnt += bisect(B, mid // A[i])
    
    # cntがKを上回るなら、そのX（変数としてはmid）は大きすぎる（Xを10**18とかとればAxBのすべてがX以下になるみたいな）
    if cnt >= K:
        hi = mid
    else:
        lo = mid
    

print(hi)
